<a href="https://colab.research.google.com/github/Rohit998/rohitdemo/blob/main/Course_evaluation_oasis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Excel writer
!pip install openpyxl

# Step 2: Upload files
from google.colab import files
uploaded = files.upload()  # Upload .csv and .xlsx

# Step 3: Import libraries
import pandas as pd

# Step 4: Load files
raw_file = "oasis_eval_export - 2025-07-08T150553.291.csv"
mapping_file = "course evaluations id match.xlsx"

raw_data = pd.read_csv(raw_file)
mapping_df = pd.read_excel(mapping_file)

# Step 5: Clean data
raw_data["Course"] = raw_data["Course"].fillna("Unknown").str.strip()
raw_data["Location"] = raw_data["Location"].fillna("Unknown").str.strip()
raw_data["Question ID"] = pd.to_numeric(raw_data["Question ID"], errors='coerce')

# Step 6: Normalize location for Fundamentals IV
fund4_mask = raw_data["Course"] == "Fundamentals of Medical Practice IV"
stjoseph_mask = raw_data["Location"].str.contains("st. joseph medical center", case=False, na=False)

raw_data.loc[fund4_mask & stjoseph_mask, "Location"] = "Mosaic Medical Center - St Joseph"
raw_data.loc[fund4_mask & ~stjoseph_mask, "Location"] = "UMKC School of Medicine"

# Step 7: Merge with mapping
merged = raw_data.merge(mapping_df, on="Question ID", how="inner")

# Step 8: Define positive labels
positive_labels = {
    "% of students who agree assessment is fair": ['Agree', 'Strongly Agree'],
    "% of students who agreed that the overall culture of the course/clerkship was respectful": ['Agree', 'Strongly Agree'],
    "% of students who reported mistreatment": ['Yes'],
    "% of students who reported workload/duty hour violations": ['No'],
    "% of students satisfied with overall quality": ['Agree', 'Strongly Agree'],
    "% of students satisfied with midterm feedback they received": ['Agree', 'Strongly Agree']

}

# Step 9: Initialize results
output_rows = []
special_courses = ["Fundamentals of Medical Practice I", "Fundamentals of Medical Practice III"]

# Step 10: Process each course
for course in merged["Course"].unique():
    course_df = merged[merged["Course"] == course]

    if course in special_courses:
        # Output as "All I"/"All III"
        grouped = course_df.groupby("Measure")
        for measure, group in grouped:
            total = len(group)
            if total == 0: continue
            element = group["Element"].iloc[0]
            labels = positive_labels.get(measure, [])
            positive = group[group["Multiple Choice Label"].isin(labels)].shape[0]
            pct = round(positive / total, 6)
            last_word = course.split()[-1]

            output_rows.append({
                "AY": "2024-25",
                "Element": element,
                "Type": "Course",
                "Phase": "Yr 3 & 4",
                "Course/Clerkship": course,
                "Campus": "All",
                "Clinical Site": f"All {last_word}",
                "Measure": measure,
                "Data": pct
            })

    else:
        # Group by location first
        grouped = course_df.groupby(["Location", "Measure"])
        for (location, measure), group in grouped:
            total = len(group)
            if total == 0: continue
            element = group["Element"].iloc[0]
            labels = positive_labels.get(measure, [])
            positive = group[group["Multiple Choice Label"].isin(labels)].shape[0]
            pct = round(positive / total, 6)

            output_rows.append({
                "AY": "2024-25",
                "Element": element,
                "Type": "Course",
                "Phase": "Yr 3 & 4",
                "Course/Clerkship": course,
                "Campus": "All",
                "Clinical Site": location,
                "Measure": measure,
                "Data": pct
            })

        # If course has multiple locations, add "All Locations"
        if course_df["Location"].nunique() > 1:
            grouped_all = course_df.groupby("Measure")
            for measure, group in grouped_all:
                total = len(group)
                if total == 0: continue
                element = group["Element"].iloc[0]
                labels = positive_labels.get(measure, [])
                positive = group[group["Multiple Choice Label"].isin(labels)].shape[0]
                pct = round(positive / total, 6)

                output_rows.append({
                    "AY": "2024-25",
                    "Element": element,
                    "Type": "Course",
                    "Phase": "Yr 3 & 4",
                    "Course/Clerkship": course,
                    "Campus": "All",
                    "Clinical Site": "All Locations",
                    "Measure": measure,
                    "Data": pct
                })

        # Add “All Practice” row for Fundamentals IV
        if course == "Fundamentals of Medical Practice IV":
            grouped_all = course_df.groupby("Measure")
            for measure, group in grouped_all:
                total = len(group)
                if total == 0: continue
                element = group["Element"].iloc[0]
                labels = positive_labels.get(measure, [])
                positive = group[group["Multiple Choice Label"].isin(labels)].shape[0]
                pct = round(positive / total, 6)

                output_rows.append({
                    "AY": "2024-25",
                    "Element": element,
                    "Type": "Course",
                    "Phase": "Yr 3 & 4",
                    "Course/Clerkship": course,
                    "Campus": "All",
                    "Clinical Site": "All Practice",
                    "Measure": measure,
                    "Data": pct
                })

# Step 11: Export to Excel
final_df = pd.DataFrame(output_rows)
output_file = "Final_Measures_by_Course_and_Location.xlsx"
final_df.to_excel(output_file, index=False)

# Step 12: Download
files.download(output_file)


KeyboardInterrupt: 